In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ir-a4-data/Reviews.csv
/kaggle/input/ir-a4-updated-data/Reviews.csv
/kaggle/input/ir-a4-updated-data/preprocessed_data.csv


In [2]:
import pandas as pd
import nltk
import re

In [3]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Load the Dataset**

In [4]:
# data = pd.read_csv("/kaggle/input/ir-a4-data/Reviews.csv")
# data

In [5]:
# data.columns

In [6]:
# data.dropna(subset = ["Text", "Summary"], inplace=True)

In [7]:
# data.shape

In [8]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string


In [9]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)


In [11]:
def preprocess_text(text):

  #convert to lowercase
  processed_txt = text.lower()

  #remove HTML tags
  processed_txt = re.sub(r'<.*?>', '', processed_txt)

  #remove non-alphanumeric chars
  processed_txt = re.sub(r'[^a-zA-Z0-9\s]', '', processed_txt)

  #remove extra whitespaces
  processed_txt = re.sub(r'\s+', ' ', processed_txt).strip()

  #tokenization
  tokens = word_tokenize(processed_txt)
  # print(tokens)

  #remove stop words and punctuations
  filtered_tokens = [word for word in tokens if word not in stop_words and word not in punctuation]
  # print(filtered_tokens)

  #convert to text by joining all filtered tokens
  processed_text = ' '.join(filtered_tokens)

  return processed_text


In [12]:
# sample_text = "Tokenization is the process of splitting text into words or smaller meaningful elements."
# preprocess_text(sample_text)

In [13]:
# data.columns

In [14]:
# data['Text'] = data['Text'].apply(preprocess_text)
# data['Summary'] = data['Summary'].apply(preprocess_text)

In [15]:
# data.columns

In [16]:
# data.shape

In [17]:
# processed_data =data[['Text', 'Summary']]
# processed_data.to_csv('preprocessed_data.csv', index=False)

In [18]:
from IPython.display import FileLink


# Create a download link for the CSV file
FileLink('preprocessed_data.csv')

/kaggle/working/preprocessed_data.csv

In [19]:
# data = pd.read_csv('/kaggle/working/preprocessed_data.csv')
# data.shape

#For Preprocessed Data
data = pd.read_csv('/kaggle/input/ir-a4-updated-data/preprocessed_data.csv')
data.shape

(568427, 2)

In [20]:
data.head()

,Text,Summary
0,bought several vitality canned dog food produc...,good quality dog food
1,product arrived labeled jumbo salted peanutsth...,advertised
2,confection around centuries light pillowy citr...,delight says
3,looking secret ingredient robitussin believe f...,cough medicine
4,great taffy great price wide assortment yummy ...,great taffy


In [21]:
new_size = int(len(data)*0.05)
print(new_size)

28421


In [22]:
# Trim the DataFrame to its 10% of original size
trimmed_data = data.sample(n=new_size, random_state=42)
trimmed_data

,Text,Summary
41432,actually tasty pure potatoes great texture nas...,like
209473,realize taste matter personal preference food ...,good subjectively 5 star
247296,one favorite cup soup choices never carried st...,lipton cup soup spring vegetable4 oz
80087,like classic taste good margarita likely youll...,suited purpose quite goal
218571,willing give chance even reading scathing revi...,tastes artificial
...,...,...
271995,doesnt amount licorice flavor aussie made prod...,good great
134443,give golden retriever two pills day thyroid si...,love pill pocketss
212846,great product pay 75 cents per pouch local gro...,overpriced
124383,coffee best aroma great medium taste replaced ...,nice medium roast


In [23]:
from sklearn.model_selection import train_test_split

In [24]:
train_data, test_data = train_test_split(trimmed_data, test_size=0.25, random_state=42)

In [25]:
print(train_data.shape)
print(test_data.shape)

(21315, 2)
(7106, 2)


In [26]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

Load the Model : GPT2

In [27]:
# Step 1: Initialize GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Custom data loaders

In [28]:
class CustomDataset(Dataset):
  def __init__(self, reviews, tokenizer, max_len=256):
    self.reviews = reviews
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.reviews)

  def __getitem__(self, idx):
    review = self.reviews.iloc[idx]['Text']
    summary = self.reviews.iloc[idx]['Summary']

    input_txt = str(review) + " <BOS> " + str(summary)
    output_txt = str(summary) + " <EOS> "

    input_ids = self.tokenizer.encode(input_txt, max_length=self.max_len, truncation=True, padding='max_length')
    output_ids = self.tokenizer.encode(output_txt, max_length=self.max_len, truncation=True, padding='max_length')

    input_ids = torch.tensor(input_ids, dtype=torch.long)
    output_ids = torch.tensor(output_ids, dtype=torch.long)

    return input_ids, output_ids

In [29]:
# Step 2: Create dataset instances

train_dataset = CustomDataset(train_data, tokenizer)
test_dataset = CustomDataset(test_data, tokenizer)

In [30]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [31]:
# Step 3: Fine-tune the GPT-2 model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [32]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

## Training

In [33]:
# Step-4 : Training

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids, labels = batch
        input_ids = input_ids.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

Epoch 1/10, Loss: 0.2209586352109909
Epoch 2/10, Loss: 0.18844905495643616
Epoch 3/10, Loss: 0.35137054324150085
Epoch 4/10, Loss: 0.0789911076426506
Epoch 5/10, Loss: 0.08545723557472229
Epoch 6/10, Loss: 0.0809001624584198
Epoch 7/10, Loss: 0.06948107481002808
Epoch 8/10, Loss: 0.07574597001075745
Epoch 9/10, Loss: 0.05396013334393501
Epoch 10/10, Loss: 0.09450507909059525


## Save the fine-tuned model

In [45]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [50]:
model_path = "fine_tuned_GPT2_model"
tokenizer_path = "fine_tuned_GPT2_tokenizer"

model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)

('fine_tuned_GPT2_tokenizer/tokenizer_config.json',
 'fine_tuned_GPT2_tokenizer/special_tokens_map.json',
 'fine_tuned_GPT2_tokenizer/vocab.json',
 'fine_tuned_GPT2_tokenizer/merges.txt',
 'fine_tuned_GPT2_tokenizer/added_tokens.json')

## Loading the Saved Model and Tokenizer

In [57]:
loaded_model = GPT2LMHeadModel.from_pretrained(model_path)
loaded_model.to(device)

loaded_tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
loaded_tokenizer.pad_token = tokenizer.eos_token

In [58]:
# Step-5: Evaluation
# model.eval()
loaded_model.eval() ##For Loaded_model
total_loss = 0.0
total_batches = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids, labels = batch
        input_ids = input_ids.to(device)
        labels = labels.to(device)
        
#         outputs = model(input_ids=input_ids, labels=labels)
        outputs = loaded_model(input_ids=input_ids, labels=labels) #Loaded_model

        loss = outputs.loss
        
        total_loss += loss.item()
        total_batches += 1

        
# Calculate average loss
print("num of batches: {}".format(total_batches))

avg_loss = total_loss / total_batches #average loss
print(f"Average Loss on Test Set: {avg_loss}")

num of batches: 889
Average Loss on Test Set: 0.21930818596626428


# Generate/Predict Summaries using the Model

In [59]:
!pip install rouge

In [60]:
from rouge import Rouge

#initialize ROUGE score
rouge = Rouge()

In [61]:
# # Initialize GPT-2 tokenizer with padding_side='left'
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2", padding_side='left')

# # Set pad_token_id to eos_token_id
# tokenizer.pad_token_id = tokenizer.eos_token_id

In [62]:
actual_summaries = []
predicted_summaries = []


#Generate summaries for the test set
# model.eval()
loaded_model.eval() ##For Loaded_model
with torch.no_grad():
    for batch in test_loader:
        input_ids, labels = batch

        input_ids = input_ids.to(device)
        
        #Generate summaries using model
        generated_ids = loaded_model.generate(input_ids=input_ids, attention_mask=input_ids.ne(tokenizer.pad_token_id),max_length=input_ids.shape[1]+2, num_beams=5, early_stopping=True)
#         print(generated_ids)
        
        #Decode generated summaries
        gen_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
#         print(len(gen_texts), gen_texts)
        
        # Get actual summaries from batch
        batch_actual_summaries = test_data.iloc[:len(gen_texts)]['Summary'].tolist()
        actual_summaries.extend(batch_actual_summaries)
        
        # Get predicted/generated summaries from batch
        predicted_summaries.extend(gen_texts)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Setting `pad_token_i

In [63]:
# print(len(actual_summaries), actual_summaries)
# print(len(predicted_summaries), predicted_summaries)

In [64]:
# Calculate ROUGE scores
scores = rouge.get_scores(predicted_summaries, actual_summaries, avg=True)
scores.keys()

dict_keys(['rouge-1', 'rouge-2', 'rouge-l'])

In [65]:
# Print ROUGE scores
print("ROUGE Scores:")
print("ROUGE-1: Precision: {:.2f}, Recall: {:.2f}, F1-Score: {:.2f}".format(scores['rouge-1']['p'], scores['rouge-1']['r'], scores['rouge-1']['f']))
print("ROUGE-2: Precision: {:.2f}, Recall: {:.2f}, F1-Score: {:.2f}".format(scores['rouge-2']['p'], scores['rouge-2']['r'], scores['rouge-2']['f']))
print("ROUGE-L: Precision: {:.2f}, Recall: {:.2f}, F1-Score: {:.2f}".format(scores['rouge-l']['p'], scores['rouge-l']['r'], scores['rouge-l']['f']))

ROUGE Scores:
ROUGE-1: Precision: 0.01, Recall: 0.06, F1-Score: 0.01
ROUGE-2: Precision: 0.00, Recall: 0.00, F1-Score: 0.00
ROUGE-L: Precision: 0.01, Recall: 0.06, F1-Score: 0.01


## Experiment with diff Hyperparameters

In [42]:
# Hyperparameters
learning_rates = [1e-5, 5e-5, 1e-4]  # list of diff learning rates
batch_sizes = [2, 4, 8]  # list of diff batch sizes
num_epochs = [3, 5, 8]  # list of diff numbers of epochs 

In [43]:
best_loss = float('inf')
# print(best_loss)

best_hyperparameters = {}
# print(best_hyperparameters)

In [44]:
# for lr in learning_rates:
#     for batch_size in batch_sizes:
#         print(lr, batch_size)
        
#         for epochs in num_epochs:
#             optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
#             train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#             test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
            
#             model.to(device)
#             model.train()
            
#             for epoch in range(epochs):
#                 total_loss = 0.0
#                 total_batches = 0
                
#                 for batch in train_loader:
#                     input_ids, labels = batch
#                     input_ids = input_ids.to(device)
#                     labels = labels.to(device)
                    
#                     optimizer.zero_grad()
                    
#                     outputs = model(input_ids=input_ids, labels=labels)
#                     loss = outputs.loss
#                     loss.backward()
#                     optimizer.step()
                    
#                     total_loss += loss.item()
#                     total_batches += 1
                    
#                 average_loss = total_loss / total_batches
#                 print(f"Epoch {epoch+1}/{epochs}, Loss: {average_loss}")
            
            
#             #Evaluate on test data
#             model.eval()
#             total_loss = 0.0
#             total_batches = 0
            
#             with torch.no_grad():
#                 for batch in test_loader:
#                     input_ids, labels = batch
#                     input_ids = input_ids.to(device)
#                     labels = labels.to(device)
                  
#                     outputs = model(input_ids=input_ids, labels=labels)
#                     loss = outputs.loss
                    
#                     total_loss += loss.item()
#                     total_batches += 1
                    
#             average_loss = total_loss / total_batches
#             print(f"Hyperparameters: LR={lr}, Batch Size={batch_size}, Epochs={epochs}, Average Loss on Test Set: {average_loss}")

#             # Check if this set of hyperparameters gives the best loss so far
#             if average_loss < best_loss:
#                 best_loss = average_loss
#                 best_hyperparameters = {'learning_rate': lr, 'batch_size': batch_size, 'num_epochs': epochs}
    
            

# print("Best Hyperparameters:", best_hyperparameters)